In [1]:
import os

list_file = sorted(list(os.walk('./file/3.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/3.part/{file}',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != '{}'].reset_index(drop=True)
input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Url'].tolist()

总数量：10


['https://www.ebay.com/itm/253139036562?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/255922672592?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/235653299500?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/255586148611?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/233621258535?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/332388018897?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256421824484?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/334041367847?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256102649999?_ul=US&_stpos=91710&orig_cvip=true',
 'https://www.ebay.com/itm/256202118103?_ul=US&_stpos=91710&orig_cvip=true']

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import json
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

df_temp, dict_, page = pd.DataFrame(), json.loads(input_.loc[a, 'Json_Compatibility']), 0
while page < dict_['page']:
    b = 0
    while True:
        b += 1
        print(b, page + 1)
        try:
            check_url = input_.loc[a, 'Url']
            
            resp = requests.post(f'''{dict_['api']}?module_groups=PART_FINDER&referrer=VIEWITEM&offset={page*20}&module=COMPATIBILITY_TABLE''',
                                 data=json.dumps(dict_['data']),
                                 headers=get_header(ua=False, **{'Content-Type': 'application/json; charset=utf-8'}),
                                 proxies=get_proxy(),
                                 timeout=(10, 10))

            if resp.status_code == 200:
                list_header = [cell['textSpans'][0]['text'].strip() for cell in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']]
                list_row = [[cell['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in cell else cell['textSpans'][0]['text'].strip() for cell in row['cells']] for row in resp.json()['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']]

                if list_header and list_row:
                    break
        except KeyboardInterrupt:
            break
        except:
            continue

    # = = = = = = = = = = = = = = =

    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                 'Item Number': input_.loc[a, 'Item Number'],
                                 'Json_Compatibility': input_.loc[a, 'Json_Compatibility'],
                                 'Page': page + 1,
                                 'Row': [i+1 for i in range(len(list_row))]})

    for i, row in enumerate(list_row):
        for j, header in enumerate(list_header):
            df_temp_temp.loc[i, header] = row[j]

    df_temp = pd.concat([df_temp, df_temp_temp], ignore_index=True).fillna('')

    # = = = = = = = = = = = = = = =

    page += 1

resp

1 1
1 2
1 3
1 4
1 5
1 6
1 7
2 7


<Response [200]>

In [8]:
df_temp.empty

False

In [9]:
output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Json_Compatibility,Page,Row,Year,Make,Model,Trim,Engine,Notes
0,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2007,Chrysler,Town & Country,Base Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
1,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,2,2007,Chrysler,Town & Country,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
2,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,3,2007,Chrysler,Town & Country,LX Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
3,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,4,2007,Chrysler,Town & Country,Limited Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
4,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,5,2007,Chrysler,Town & Country,Touring Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
...,...,...,...,...,...,...,...,...,...,...,...
127,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,8,2001,Chrysler,Voyager,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 FLEX OHV Naturally A...,
128,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,9,2001,Chrysler,Voyager,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
129,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,10,2001,Dodge,Caravan,SE Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 FLEX OHV Naturally A...,
130,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,11,2001,Dodge,Caravan,SE Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,


In [10]:
crawler_status = 'ok'

crawler_status

'ok'

In [11]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
    df_temp.loc[0, 'Check_Url'] = check_url
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [12]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No', 'Page', 'Row'],
                                                ascending=[True, True, True],
                                                ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,No,Item Number,Json_Compatibility,Page,Row,Year,Make,Model,Trim,Engine,Notes
0,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,1,2007,Chrysler,Town & Country,Base Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
1,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,2,2007,Chrysler,Town & Country,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
2,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,3,2007,Chrysler,Town & Country,LX Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
3,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,4,2007,Chrysler,Town & Country,Limited Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
4,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",1,5,2007,Chrysler,Town & Country,Touring Mini Passenger Van 4-Door,3.8L 3778CC 231Cu. In. V6 GAS OHV Naturally As...,
...,...,...,...,...,...,...,...,...,...,...,...
127,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,8,2001,Chrysler,Voyager,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 FLEX OHV Naturally A...,
128,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,9,2001,Chrysler,Voyager,LX Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,
129,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,10,2001,Dodge,Caravan,SE Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 FLEX OHV Naturally A...,
130,1,253139036562,"{""api"": ""https://www.ebay.com/g/api/finders"", ...",7,11,2001,Dodge,Caravan,SE Mini Passenger Van 4-Door,3.3L 3301CC 201Cu. In. V6 GAS OHV Naturally As...,


In [13]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
